# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import numpy as np
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# Delete and Create all tables before re-running this ETL experiment
import create_tables
create_tables.main()

In [3]:
import numpy

# numpy datatypes registered into psycopg2
from psycopg2.extensions import register_adapter, AsIs
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(numpy.float64, addapt_numpy_float64)
register_adapter(numpy.int64, addapt_numpy_int64)

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = (get_files(filepath='data/song_data/A/A/A') + 
              get_files(filepath='data/song_data/A/A/B') + 
              get_files(filepath='data/song_data/A/A/C') +
              get_files(filepath='data/song_data/A/B/A') +
              get_files(filepath='data/song_data/A/B/B') +
              get_files(filepath='data/song_data/A/B/C')
             )
song_files[:10]

['/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/wb/gitrepos/udacity_project_data_model

In [7]:
def build_df_from_multiple_filepaths(filepaths):
    """Read multiple filepaths and turn them into a pd.DataFrame.
    Also, replace empty strings and np.nan to None"""
    
    dfs = [pd.read_json(file, lines=True) for file in filepaths]
    dfs = pd.concat(dfs, ignore_index=True)
    
#     To keep data types consistent, first we will replace all np.nan and empty strings with None
    dfs.replace(to_replace="", value=np.nan, inplace=True)
    dfs = dfs.where(cond=pd.notnull(dfs), other=None)
    
    return dfs

In [8]:
df_songs = build_df_from_multiple_filepaths(filepaths=song_files)
df_songs.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR8ZCNI1187B9A069B,None,None,None,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
1,1,ARKFYS91187B98E58F,None,None,None,Jeff And Sheri Easter,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),267.7024,0
2,1,AR7G5I41187FB4CE6C,None,None,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
3,1,ARXR32B1187FB57099,None,None,None,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
4,1,ARD0S291187B9B7BF5,None,None,Ohio,Rated R,SOMJBYD12A6D4F8557,Keepin It Real (Skit),114.78159,0


In [9]:
df_songs.shape

(71, 10)

## #1: `artists` Table
---
This table must be created first since it has no foreign key dependency.

#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df_songs["artist_id"].nunique()

69

In [11]:
df_artist = df_songs[["artist_id",
                      "artist_name",
                      "artist_location",
                      "artist_latitude",
                      "artist_longitude"]]\
                .dropna(axis=0, subset=["artist_id"])

df_artist.drop_duplicates(subset="artist_id", inplace=True)
df_artist.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR8ZCNI1187B9A069B,Planet P Project,None,None,None
1,ARKFYS91187B98E58F,Jeff And Sheri Easter,None,None,None
2,AR7G5I41187FB4CE6C,Adam Ant,"London, England",None,None
3,ARXR32B1187FB57099,Gob,None,None,None
4,ARD0S291187B9B7BF5,Rated R,Ohio,None,None


In [12]:
df_artist.shape

(69, 5)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
for i, row in df_artist.iterrows():
    cur.execute(artist_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [14]:
df_songs_table = df_songs[["song_id",
                           "title",
                           "artist_id",
                           "year",
                           "duration"]]\
                .dropna(axis=0, subset=["song_id"])

df_songs.drop_duplicates(subset="song_id", inplace=True)
df_songs_table.head()

,song_id,title,artist_id,year,duration
0,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),ARKFYS91187B98E58F,0,267.7024
2,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
3,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
4,SOMJBYD12A6D4F8557,Keepin It Real (Skit),ARD0S291187B9B7BF5,0,114.78159


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [15]:
for i, row in df_songs_table.iterrows():
    cur.execute(song_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files_paths = get_files(filepath='data/log_data/2018/11')
log_files_paths

['/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-13-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-12-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-17-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-14-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-11-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-03-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-05-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-19-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-26-events.json',
 '/home/wb/gitrepos/udacity_project_data_model

In [17]:
df_log = build_df_from_multiple_filepaths(filepaths=log_files_paths)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,None,free,"Harrisburg-Carlisle, PA",GET,Home,1540006905796.0,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796.0,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,None,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1540676534796.0,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1540676534796.0,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1540872073796.0,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9


In [18]:
df_log.shape

(7302, 18)

In [19]:
# Is there any song intersection between datasets?
songs_intersection = set(df_log["song"]).intersection(set(df_songs["title"]))
songs_intersection

{'Intro', 'Setanta matins'}

In [20]:
# Is there any artists intersection between datasets?
artists_intersection = set(df_log["artist"]).intersection(set(df_songs["artist_name"]))
artists_intersection

{'Blue Rodeo',
 'Elena',
 'Gob',
 'Gwen Stefani',
 'Jimmy Wakely',
 'Line Renaud',
 'Lionel Richie',
 'Lupe Fiasco',
 'Sophie B. Hawkins',
 'Tom Petty',
 'Trafik'}

In [21]:
df_log.loc[df_log["artist"].isin(list(artists_intersection)), ["artist", "song", "length"]]

,artist,song,length
687,Lionel Richie,Hello,249.62567
1069,Lupe Fiasco,Shining Down [feat. Matthew Santos] (Amended A...,273.94567
1184,Gwen Stefani,Danger Zone,216.76363
1297,Jimmy Wakely,I Love You So Much It Hurts,165.74649
2042,Gob,Ming Tran,154.17424
2092,Tom Petty,Runnin' Down A Dream,263.23546
2125,Tom Petty,Runnin' Down A Dream,263.23546
2391,Trafik,Dirty Word,380.21179
2750,Lupe Fiasco,Shining Down [feat. Matthew Santos] (Amended A...,273.94567
2751,Tom Petty,Runnin' Down A Dream,263.23546


In [22]:
df_songs.loc[df_songs["artist_name"].isin(list(artists_intersection)), ["artist_name", "title", "duration"]]

,artist_name,title,duration
3,Gob,Face the Ashes,209.60608
12,Blue Rodeo,Floating,491.12771
16,Jimmy Wakely,Broken-Down Merry-Go-Round,151.84934
20,Lionel Richie,Tonight Will Be Alright,307.3824
29,Elena,Setanta matins,269.58322
43,Trafik,Salt In NYC,424.12363
54,Tom Petty,A Higher Place (Album Version),236.17261
62,Gwen Stefani,Harajuku Girls,290.55955
67,Lupe Fiasco,Streets On Fire (Explicit Album Version),279.97995
68,Line Renaud,Der Kleine Dompfaff,152.92036


In [23]:
import sys
sys.exit()

SystemExit: 

/home/wb/gitrepos/udacity_project_data_modeling_postgres/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
# Filtering records by NextSong action, Home action doesnt have a song registered
df_log = df_log[df_log["page"]=="NextSong"]
df_log.reset_index(inplace=True, drop=True)

In [ ]:

timestamp = pd.to_datetime(df_log["ts"], unit='ms').values
hour = pd.to_datetime(df_log["ts"], unit='ms').dt.hour.values
day = pd.to_datetime(df_log["ts"], unit='ms').dt.day.values
week = pd.to_datetime(df_log["ts"], unit='ms').dt.isocalendar().week.values
month = pd.to_datetime(df_log["ts"], unit='ms').dt.month.values
year = pd.to_datetime(df_log["ts"], unit='ms').dt.year.values
weekday = pd.to_datetime(df_log["ts"], unit='ms').dt.weekday.values

column_labels = ("start_time", "hour", "day", "week", "month", "year", "weekday")
time_data = (timestamp, hour, day, week, month, year, weekday)

df_time = pd.DataFrame(dict(zip(column_labels, time_data)))
df_time.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in df_time.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
df_user = df_log[["userId",
                  "firstName",
                  "lastName",
                  "gender",
                  "level"]]\
                .dropna(axis=0, subset=["userId"])

df_user["userId"] = df_user["userId"].astype(int)
df_user.drop_duplicates(subset="userId", inplace=True)
df_user.head(1)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
df_user.head(5)

In [ ]:
for i, row in df_user.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
#     cur.execute(song_select, (row.song, row.artist, row.length))
    cur.execute(song_select, (row.song, row.artist))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    if songid:
        print(f"songname: {row.song}, songid:{songid}, artistid: {artistid}")
    elif artistid:
        print(f"songname: {row.song}, songid:{songid}, artistid: {artistid}")
        
    # insert songplay record
    timestamp = pd.to_datetime(row.ts, unit='ms')
    songplay_data = (timestamp,
                     row.userId,
                     row.level,
                     songid,
                     artistid,
                     row.sessionId,
                     row.location,
                     row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [ ]:
# import sys
# sys.exit()

# cur.close()
# conn.close()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
cur.close()
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.